In [ ]:
#!pip install tweepy
#!pip install snscrape

In [7]:
import os
import datetime
import time
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
##from api_access import api
import tweepy as tw
import re

In [3]:
import nltk
from nltk.corpus import stopwords
stop_nltk = stopwords.words("english")
from wordcloud import WordCloud

ModuleNotFoundError: No module named 'wordcloud.query_integral_image'

In [10]:
api

In [9]:
# Credentials

consumer_key = "RoYF5ReN9QYTumXTr5MBF3G5p"
consumer_secret = "VZFvLXCw6TdsQxl1TW0gh9NeZcVqJ5gdIP3V0ijrfBaFvqbnrq"
access_token = "146918630-7Bkih63wYnHWVvScTqGBRkhCr5ccm022mOQ6FpIQ"
access_token_secret = "qh9mkqsXUAnUdB2BJrHekDmEpYyyR4PWSbVGiM3jvGssW"

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth,wait_on_rate_limit=True)

In [22]:
def get_tweet_ids(keywords,maxTweets,startDate,endDate,fname):
    
    '''
    Function to scrape tweet ids using snscrape 
    '''
    for i in keywords:
        f = os.system('snscrape --max-results '+str(maxTweets)+' twitter-search '+'"'+i+' since:'+startDate+' until:'+endDate+'"'+' >fetched_tweet_ids_'+fname+'.txt')
        if f==0:
            print(f"Successfully fetched {maxTweets} tweet ids!")

In [23]:
def fetch_tw(ids,fname):
    
    '''
    Function to extract tweet attributes - tweet id, tweet, date
    '''
    list_of_tw_status = api.statuses_lookup(ids,tweet_mode="extended")
    empty_data = pd.DataFrame()
    for status in list_of_tw_status:
        if status.lang=='en': #getting only english tweets
            tweet_elem = {"date":status.created_at,
                          "tweet":status.full_text}
            empty_data = empty_data.append(tweet_elem,ignore_index = True)
    empty_data.to_csv("fetched_tweets_"+fname+".csv", mode="a")

In [27]:
def get_tweets(fname):
    
    '''
    Function to get tweet data using tweet ids
    '''
    #Extracting ids from the tweet urls
    tweet_url = pd.read_csv("fetched_tweet_ids_"+fname+".txt", index_col= None, header = None, names = ["links"])
    af = lambda x: x["links"].split("/")[-1]
    tweet_url['id'] = tweet_url.apply(af, axis=1)
    ids = tweet_url['id'].tolist()
    
    # Processing the ids in batches
    total_count = len(ids)
    chunks = (total_count - 1) // 50 + 1
    for i in range(chunks):
        batch = ids[i*50:(i+1)*50]
        result = fetch_tw(batch,fname)
        print(f'Batch {i+1} processed successfully...')

In [14]:
def preprocess(startDate,endDate,fname):

    tweet_data = pd.read_csv('fetched_tweets_'+fname+'.csv',usecols=['date','tweet'])
    tweet_data.columns = ['Date_time','Tweet_text']

    # Removing repetitive headers
    indices = tweet_data[tweet_data['Tweet_text']=='tweet'].index
    tweet_data.drop(indices,inplace=True)

    # Converting to datetime object
    tweet_data[['Date','Time']] = tweet_data['Date_time'].str.split(" ",expand=True)
    tweet_data = tweet_data.drop(['Date_time','Time'],axis=1)
    tweet_data['Date'] = pd.to_datetime(tweet_data['Date'],infer_datetime_format=True)

    # Removing rows that fall outside of the date range
    tweet_data = tweet_data[(tweet_data['Date']>=startDate) & (tweet_data['Date']<=endDate)]

    return tweet_data[['Date','Tweet_text']]

In [28]:
keywords = ['#SuzukiIndia', '@suzuki2wheelers'] 
maxTweets = 30000
dates = ['2019-07-01','2020-02-29','2020-03-01',str(datetime.date.today())]
periods = ['precovid','postcovid']

In [58]:
get_tweet_ids(keywords,maxTweets,dates[2],dates[3],periods[1])

Successfully fetched 30000 tweet ids!
Successfully fetched 30000 tweet ids!


In [59]:
get_tweets(periods[1])

Batch 1 processed successfully...
Batch 2 processed successfully...
Batch 3 processed successfully...
Batch 4 processed successfully...
Batch 5 processed successfully...
Batch 6 processed successfully...
Batch 7 processed successfully...
Batch 8 processed successfully...
Batch 9 processed successfully...
Batch 10 processed successfully...
Batch 11 processed successfully...
Batch 12 processed successfully...
Batch 13 processed successfully...
Batch 14 processed successfully...
Batch 15 processed successfully...
Batch 16 processed successfully...
Batch 17 processed successfully...
Batch 18 processed successfully...
Batch 19 processed successfully...
Batch 20 processed successfully...
Batch 21 processed successfully...
Batch 22 processed successfully...
Batch 23 processed successfully...
Batch 24 processed successfully...
Batch 25 processed successfully...
Batch 26 processed successfully...
Batch 27 processed successfully...
Batch 28 processed successfully...
Batch 29 processed successful

In [60]:
pre_covid_tweets = preprocess(dates[2],dates[3],periods[1])
pre_covid_tweets

,Date,Tweet_text
0,2020-10-12,@SudheeshSRao @suzuki2wheelers Point taken.. ✌️
1,2020-10-12,@suzuki2wheelers Beautiful gift for @suzuki2wh...
2,2020-10-14,"@KaifraVel Dear Sir,\n\nWe regret the inconven..."
3,2020-10-12,"@dheerajkalra1 Dear Dheeraj,\n\nWe regret the ..."
4,2020-10-14,@tvsmotorcompany @BAJAJ @suzuki2wheelers @Hero...
...,...,...
3690,2020-03-02,"@itheprasad Dear Sir,\n\nThank you for taking ..."
3691,2020-03-02,When @suzuki2wheelers called us for the Media ...
3692,2020-03-02,"@Vishwatmaa1 Dear Sir,\n\nThank you for writin..."
3693,2020-03-02,"@AsiyaSh26148781 Dear Sir,\n\nWe are concerned..."


In [61]:
pre_covid_tweets['CLean Tweet_text']=pre_covid_tweets['Tweet_text'].apply(clean_tweet)

In [63]:
pre_covid_tweets

,Date,Tweet_text,CLean Tweet_text,sentiment
0,2020-10-12,@SudheeshSRao @suzuki2wheelers Point taken.. ✌️,point taken,Neutral
1,2020-10-12,@suzuki2wheelers Beautiful gift for @suzuki2wh...,beautiful gift for,Positive
2,2020-10-14,"@KaifraVel Dear Sir,\n\nWe regret the inconven...",dear sir regret the inconvenience caused reque...,Positive
3,2020-10-12,"@dheerajkalra1 Dear Dheeraj,\n\nWe regret the ...",dear dheeraj regret the inconvenience caused r...,Positive
4,2020-10-14,@tvsmotorcompany @BAJAJ @suzuki2wheelers @Hero...,what think about kerala customer why want show...,Negative
...,...,...,...,...
3690,2020-03-02,"@itheprasad Dear Sir,\n\nThank you for taking ...",dear sir thank you for taking the time write a...,Neutral
3691,2020-03-02,When @suzuki2wheelers called us for the Media ...,when called for the media endurance race sent ...,Positive
3692,2020-03-02,"@Vishwatmaa1 Dear Sir,\n\nThank you for writin...",dear sir thank you for writing would request y...,Positive
3693,2020-03-02,"@AsiyaSh26148781 Dear Sir,\n\nWe are concerned...",dear sir are concerned hear your dissatisfacti...,Positive


In [64]:
pre_covid_tweets.to_csv("fetched_clean_tweets_postcovid.csv", mode="a")

In [36]:
##Let's also write our sentiment analyzer function:

from textblob import TextBlob
def analyze_sentiment(tweet):
    analysis = TextBlob(tweet)
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity ==0:
        return 'Neutral'
    else:
        return 'Negative'

In [62]:
pre_covid_tweets['sentiment']=pre_covid_tweets['Tweet_text'].apply(lambda x:analyze_sentiment(x))

In [32]:
def clean_tweet(tweet):
    
    '''
    Function to clean tweet text
    '''
    # Removing links,special characters, etc. using regular expressions
    text = re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|(#[A-Za-z0-9]+)", " ",tweet)
    # Removing words like rt and search string
    ##drop_list = ["suzuki"]
    tokens = text.lower().strip().split()
    tokens_clean = [term for term in tokens if term not in drop_list and len(term)>2]
    return ' '.join(tokens_clean)

In [47]:
pre_covid_tweets['covid'] = [txt for txt in pre_covid_tweets['Tweet_text'].values if re.findall("covid", txt)]
pre_covid_tweets

ValueError: Length of values does not match length of index

In [46]:
import re

In [56]:
ttt=[]
for txt in pre_covid_tweets['Tweet_text'].values:
    if re.findall("covid", txt):
        print(txt)
    else:
        ttt.append(txt)

In [57]:
ttt

['INFO PROMO &amp; TEST DRIVE SUZUKI\nhttps://t.co/p3g49gvtYx\nfast respon 08 ##suzukibandung #suzuki #danisuzukibandung #promosuzuki #suzukibandungco',
 'CeativeBikers - Motion in Pleasure. Get your gear @ https://t.co/XhPEGdDEA2 Tag and Share.\n#helmet #Motorcycle #ducati #bikelife #moto #motosport #motocross #superbike #yamaha #kawasaki #suzuki #harleydavidson #bikergirl #girlsbiker https://t.co/xY9kZXAEzK',
 '#FBF to the original 1982 Katana with some detail photos of the original machine.\n---\n#Suzuki #SuzukiCycles #KATANA https://t.co/1ybvTXtdGM',
 "Before we cross the bridge into a new month, let's make this leap year a count. Padmaja Suzuki wishes everyone a happy leap day!!\n\n#padmajasuzuki #suzuki #suzukiaccess125 #access125 #gixxersf250 #intruder #burgman #hayabusa #leapday #leapyear #Vijayawada #andhrapradesh https://t.co/7MacEo8U9Y",
 "Thinking about buying #chinese motorcycle fairings but wondering if I'll ever get then due to #coronavirus \n#motorcycles #suzuki #katana

In [ ]:
pre_covid_clean_tweets = [clean_tweet(txt) for txt in pre_covid_tweets['Tweet_text'].values]
pre_covid_clean_tweets
re.findall("\S+@(\S+)", text)

In [296]:
pre_covid_comb_tweets = " ".join(pre_covid_clean_tweets)
pre_covid_comb_tweets[:500]

'recently one the meteor prototypes was caught camera while was out test for nvh levels now new development four prototypes the upcoming have been spied parking lot the meteor has been spotted once again without any camouflage when will get see the the flesh head here know how much love the long evening shadows captured beautifully shenoy the himalayan ridepure motorcycletrips motorcycleride motorcyclediaries motorcyclesofinstagram chasingthesun chasingadv acrylic keychain numberplatekeychain cus'